<a href="https://colab.research.google.com/github/TOKTAK007/Image-Segment/blob/main/Copy_of_VGG_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
!pip install torch_summary
from torchsummary import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


torchsummary เป็นเครื่องมือ (tool) สำหรับการสรุปโครงสร้างของโมเดลประสาทเทียม (neural network) ใน PyTorch ที่ช่วยให้ผู้ใช้สามารถตรวจสอบขนาดของแต่ละชั้น (layer) และจำนวนพารามิเตอร์ (parameters) ในโมเดลได้อย่างง่ายดาย โดยใช้ฟังก์ชัน summary ซึ่งเป็นฟังก์ชันที่สร้างขึ้นมาเพื่อให้ผู้ใช้สามารถสรุปโครงสร้างของโมเดลได้อย่างรวดเร็ว



torchvision เป็นชุดเครื่องมือ (toolkit) สำหรับการประมวลผลภาพ (image processing) ใน PyTorch ซึ่งช่วยให้ผู้ใช้สามารถทำงานกับข้อมูลภาพได้อย่างสะดวกและรวดเร็วขึ้น ซึ่งได้รวมถึงการโหลด (loading) และการประมวลผลภาพต่างๆ เช่น การแปลงขนาด (resizing), การทำซ้อนภาพ (image overlay), การแสดงผล (visualization) และอื่นๆ อีกมากมาย นอกจากนี้ torchvision ยังมีโมดูลสำหรับการประมวลผลข้อมูลวิดีโอ (video processing) อีกด้วย

torch.nn.functional เป็นโมดูลของ PyTorch ที่มีฟังก์ชัน (function) ต่างๆ ที่ใช้ในการประมวลผลข้อมูลในโมเดลประสาทเทียม (neural network) เช่น การคำนวณค่าเสีย (loss), การประมวลผลทางคณิตศาสตร์ (mathematical operations) และการประมวลผลข้อมูลในชั้นต่างๆ ของโมเดลประสาทเทียม เช่น การทำคอนโวลูชัน (convolution), การทำประสาทเทียมแบบลูกถ้วย (LSTM), การทำเอฟเฟกต์ (effect) และอื่นๆ อีกมากมาย

การ import torch.nn.functional as F จึงเป็นการ import โมดูล functional ของ torch.nn และกำหนดชื่อตัวแปรให้เป็น F เพื่อที่จะเข้าถึงฟังก์ชันต่างๆ ในโมดูลนี้ได้อย่างสะดวก โดยจะใช้ F แทน torch.nn.functional ในการเรียกใช้ฟังก์ชันต่างๆ ที่อยู่ใน functional นั้น


In [2]:
model = models.vgg16(pretrained=True).to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 122MB/s]


โค้ด model = models.vgg16(pretrained=True).to(device) ใช้โมเดล VGG-16 ที่ถูกสร้างด้วยไลบรารี Torchvision ใน PyTorch โดยกำหนดให้โมเดลนี้โหลดน้ำหนักก่อน (pretrained) จากชุดข้อมูล ImageNet และนำมาใช้ในการคำนวณผ่าน CPU/GPU ซึ่งเราสามารถระบุได้ด้วย device ที่กำหนดไว้

โมเดล VGG-16 เป็นโมเดลประสาทเทียมที่มีความลึก 16 ชั้น (layers) และถูกออกแบบมาเพื่อใช้ในงานปัญญาประดิษฐ์ที่เกี่ยวข้องกับการจำแนกหมวดหมู่ภาพ โดยสามารถใช้งานได้ทั้งในการทำ Transfer Learning และ Fine-tuning กับชุดข้อมูลภาพต่าง ๆ ที่มีความซับซ้อนต่างกัน

โค้ด to(device) จะนำโมเดล VGG-16 มาเรียกใช้บน CPU/GPU ที่กำหนดไว้ด้วย device ซึ่งทำให้เราสามารถใช้งานโมเดลได้เร็วขึ้นและสามารถปรับใช้กับข้อมูลที่มีขนาดใหญ่ได้มากขึ้น โดยที่ไม่เกิดปัญหาเกี่ยวกับ Memory ขณะที่ทำงานกับชุดข้อมูลที่มีขนาดใหญ่

=====================================

โดยในตัวอย่างข้างต้น torch.zeros((3, 4)) จะสร้าง Tensor 2 มิติที่มีค่าเป็น 0 ทั้งหมด ขนาด 3x4 และ torch.zeros((2, 3, 4), dtype=torch.int) จะสร้าง Tensor 3 มิติที่มีค่าเป็น 0 ทั้งหมด ขนาด 2x3x4 

In [3]:
summary(model, torch.zeros(1,3,224,224));

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 7, 7]           --
|    └─Conv2d: 2-1                       [-1, 64, 224, 224]        1,792
|    └─ReLU: 2-2                         [-1, 64, 224, 224]        --
|    └─Conv2d: 2-3                       [-1, 64, 224, 224]        36,928
|    └─ReLU: 2-4                         [-1, 64, 224, 224]        --
|    └─MaxPool2d: 2-5                    [-1, 64, 112, 112]        --
|    └─Conv2d: 2-6                       [-1, 128, 112, 112]       73,856
|    └─ReLU: 2-7                         [-1, 128, 112, 112]       --
|    └─Conv2d: 2-8                       [-1, 128, 112, 112]       147,584
|    └─ReLU: 2-9                         [-1, 128, 112, 112]       --
|    └─MaxPool2d: 2-10                   [-1, 128, 56, 56]         --
|    └─Conv2d: 2-11                      [-1, 256, 56, 56]         295,168
|    └─ReLU: 2-12                        [-1, 256, 56, 56]      

summary(model, torch.zeros(1,3,224,224)) เป็นการเรียกฟังก์ชัน summary ซึ่งเป็นฟังก์ชันที่ช่วยสรุปข้อมูลของโมเดลประสาทเทียมและจำนวนพารามิเตอร์ทั้งหมดในโมเดล โดยรับอินพุตเป็นโมเดลประสาทเทียม (model) และตัวอย่างข้อมูล (input) ที่ต้องการส่งเข้าไปในโมเดลเพื่อคำนวณผลลัพธ์

ในที่นี้ model คือโมเดล VGG-16 ที่ถูกสร้างขึ้นด้วยไลบรารี Torchvision และ torch.zeros(1,3,224,224) คือตัวอย่างข้อมูลภาพที่มีขนาด 1 ชุด (batch size) และมีขนาดภาพเป็น 3x224x224 (ความสูง 224 พิกเซล ความกว้าง 224 พิกเซล และมีช่องสี RGB 3 ช่อง)

ผลลัพธ์ที่ได้จากการเรียกใช้ summary(model, torch.zeros(1,3,224,224)) คือสรุปข้อมูลของโมเดล VGG-16 ที่เราสร้างขึ้น ได้แก่จำนวนชั้น (layers) ทั้งหมด, จำนวนพารามิเตอร์ (parameters) ทั้งหมด, และขนาดของ Tensor ที่เป็น Output ของแต่ละชั้น (Output Shape) เพื่อช่วยในการตรวจสอบและปรับแต่งโมเดลให้เหมาะสมกับงานที่ต้องการใช้งาน

In [4]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Conv2d: 2-1 [-1, 64, 224, 224] คือ output shape หรือขนาดของ Tensor ที่ผ่านการผ่าน Convolutional Layer แบบ 2 มิติ (2D Convolution) ด้วย filter หรือ kernel ที่มีขนาดเท่ากับ 3x3 และมี stride เท่ากับ 1 (คือขนาดของ stride คือ 1x1) โดย Input Tensor จะมีขนาดเท่ากับ [-1, 3, 224, 224] และมี batch size ไม่กำหนด (เป็น -1) ซึ่งในที่นี้จะถูกส่งเข้าไปในโมเดล CNN แบบ VGG16 ที่มี pretrained weights โดย output Tensor จะมีขนาด 64 ช่อง (channel) และขนาดของภาพเท่ากับ 224x224 พร้อมกับ batch size เดียวกับ Input Tensor โดย [-1, 64, 224, 224] จะหมายความว่า batch size ยังคงเป็นตัวเดียวกับ Input Tensor ซึ่งจะนำไปผ่าน layer ต่อไปในโมเดล VGG16 ได้เรื่อยๆ จนกระทั่งเอาไปใช้ในการทำนายหรือในการเรียนรู้แบบ Supervised Learning



